Kural Tabanlı Sınıflandırma ile Potansiyel Müşteri Getirisi Hesaplama

 İş Problemi

 Gezinomi yaptığı satışların bazı özelliklerini kullanarak seviye tabanlı (level based) yeni satış tanımları
 oluşturmak ve bu yeni satış tanımlarına göre segmentler oluşturup bu segmentlere göre yeni gelebilecek müşterilerin şirkete
 ortalama ne kadar kazandırabileceğini tahmin etmek istemektedir.
 Örneğin: Antalya’dan Herşey Dahil bir otele yoğun bir dönemde gitmek isteyen bir müşterinin
        ortalama ne kadar kazandırabileceği belirlenmek isteniyor.

In [1]:
# PROJE GÖREVLERİ
# GÖREV 1: Aşağıdaki soruları yanıtlayınız.
# Soru 1: miuul_gezinomi.xlsx dosyasını okutunuz ve veri seti ile ilgili genel bilgileri gösteriniz.
import pandas as pd
pd.set_option("display.max_columns", None)
df=pd.read_excel("miuul_gezinomi.xlsx")

In [2]:
#genel analiz fonksiyonu
def check_df(dataframe, head=5):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(head))
    print("##################### Tail #####################")
    print(dataframe.tail(head))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### Quantiles #####################")
    print(dataframe.describe([0, 0.05, 0.50, 0.95, 0.99, 1]).T)
check_df(df)

##################### Shape #####################
(59164, 9)
##################### Types #####################
SaleId                         int64
SaleDate              datetime64[ns]
CheckInDate           datetime64[ns]
Price                        float64
ConceptName                   object
SaleCityName                  object
CInDay                        object
SaleCheckInDayDiff             int64
Seasons                       object
dtype: object
##################### Head #####################
   SaleId   SaleDate CheckInDate       Price     ConceptName SaleCityName  \
0  415122 2022-12-03  2022-12-03   79.304029    Herşey Dahil      Antalya   
1  415103 2022-12-03  2022-12-03   45.970696  Yarım Pansiyon      Antalya   
2  404034 2022-09-12  2022-09-13   77.838828    Herşey Dahil      Antalya   
3  415094 2022-12-03  2022-12-10  222.710623  Yarım Pansiyon        İzmir   
4  414951 2022-12-01  2022-12-03  140.476190  Yarım Pansiyon        İzmir   

     CInDay  SaleCheckInDayDif

In [3]:
#Soru 2:Kaçunique şehirvardır? Frekanslarınedir?
df["SaleCityName"].nunique()

6

In [4]:
df["SaleCityName"].value_counts()

Antalya    31649
Muğla      10662
Aydın      10646
Diğer       3245
İzmir       2507
Girne        455
Name: SaleCityName, dtype: int64

In [5]:
#Soru 3:Kaç unique Concept vardır?
df["ConceptName"].nunique()            

3

In [6]:
#Soru4: Hangi Concept’den kaçar tane satış gerçekleşmiş?
df["ConceptName"].value_counts()

Herşey Dahil      53186
Yarım Pansiyon     3559
Oda + Kahvaltı     2419
Name: ConceptName, dtype: int64

In [7]:
#Soru5: Şehirlere göre satışlardan toplam ne kadar kazanılmış?
#ücret toplamı e olmaması için
pd.set_option('display.float_format', lambda x: '%.2f' % x)

df.groupby("SaleCityName").agg({"Price":"sum"})

,Price
SaleCityName,
Antalya,2041911.10
Aydın,573296.01
Diğer,154572.29
Girne,27065.03
Muğla,665842.21
İzmir,165934.83


In [8]:
#Soru6: Concept türlerine göre göre ne kadar kazanılmış?
df.groupby("ConceptName").agg({"Price":"sum"})

,Price
ConceptName,
Herşey Dahil,3332910.77
Oda + Kahvaltı,121308.35
Yarım Pansiyon,174402.35


In [9]:
#Soru7: Şehirlere göre PRICE ortalamaları nedir?
df.groupby("SaleCityName").agg({"Price":"mean"})
#df.groupby(by=['SaleCityName']).agg({"Price": "mean"})


,Price
SaleCityName,
Antalya,64.52
Aydın,53.86
Diğer,47.71
Girne,59.48
Muğla,62.46
İzmir,66.27


In [10]:
#Soru 8: Conceptlere göre PRICE ortalamaları nedir?
df.groupby("ConceptName").agg({"Price":"mean"})

,Price
ConceptName,
Herşey Dahil,62.67
Oda + Kahvaltı,50.25
Yarım Pansiyon,49.03


In [11]:
#Soru 9: Şehir-Concept kırılımında PRICE ortalamalarınedir?
df.groupby(["SaleCityName","ConceptName"]).agg({"Price":"mean"})

Price
SaleCityName ConceptName          
Antalya      Herşey Dahil    64.52
             Oda + Kahvaltı  63.50
             Yarım Pansiyon  67.19
Aydın        Herşey Dahil    54.00
             Oda + Kahvaltı  34.46
             Yarım Pansiyon  30.02
Diğer        Herşey Dahil    84.77
             Oda + Kahvaltı  37.60
             Yarım Pansiyon  42.11
Girne        Herşey Dahil    97.68
             Oda + Kahvaltı  39.78
             Yarım Pansiyon  53.25
Muğla        Herşey Dahil    63.02
             Oda + Kahvaltı  59.04
             Yarım Pansiyon  45.12
İzmir        Herşey Dahil    74.70
             Oda + Kahvaltı  41.32
             Yarım Pansiyon  59.61

Görev 2: SaleCheckInDayDiff değişkenini kategorik bir değişkene çeviriniz.


In [12]:
#önce aralıklarla label belirlemek
bins=[1,7,30,90,df["SaleCheckInDayDiff"].max()]
labels=["Last Minuters", "Potential Planners", "Planners", "Early Bookers"]

In [13]:
#yeni değişkene atama yaparken bölnecek değişken, aralıklar , etiketler
df["EB_Score"]=pd.cut(df["SaleCheckInDayDiff"],bins, labels=labels)

In [14]:
df.head(50).to_excel("eb_score.xlsx",index=False)

 GÖREV 3: Şehir,Concept, [EB_Score,Sezon,CInday] kırılımında ücret ortalamalarına ve frekanslarına bakınız

In [15]:
# Şehir-Concept-EB Score kırılımında ücret ortalamaları
df.groupby(["SaleCityName","ConceptName","EB_Score"]).agg({"Price": ["mean","count"]})

Price      
                                                mean count
SaleCityName ConceptName    EB_Score                      
Antalya      Herşey Dahil   Last Minuters      61.01  8187
                            Potential Planners 64.90  8874
                            Planners           67.88  4490
                            Early Bookers      66.49  3281
             Oda + Kahvaltı Last Minuters      62.46   156
...                                              ...   ...
İzmir        Oda + Kahvaltı Early Bookers      66.93     3
             Yarım Pansiyon Last Minuters      53.77   372
                            Potential Planners 64.80   198
                            Planners           60.51    47
                            Early Bookers      65.91    14

[72 rows x 2 columns]

In [16]:
# Şehir-Concept-Sezon kırılımında ücret ortalamaları
df.groupby(["SaleCityName","ConceptName","Seasons"]).agg({"Price": ["mean","count"]})

Price       
                                      mean  count
SaleCityName ConceptName    Seasons              
Antalya      Herşey Dahil   High     64.92  27126
                            Low      61.55   3667
             Oda + Kahvaltı High     66.27    303
                            Low      60.67    295
             Yarım Pansiyon High     73.26    118
                            Low      62.00    138
Aydın        Herşey Dahil   High     54.95  10103
                            Low      33.68    473
             Oda + Kahvaltı High     30.39     27
                            Low      44.45     11
             Yarım Pansiyon High     32.63     20
                            Low      25.27     11
Diğer        Herşey Dahil   High     83.79    364
                            Low      87.31    141
             Oda + Kahvaltı High     34.84    349
                            Low      39.96    408
             Yarım Pansiyon High     39.73    516
                            Low      42.95   1462
Girne        Herşey Dahil   High    103.94     55
                            Low      90.94     51
             Oda + Kahvaltı High     39.48     66
                            Low      40.04     73
             Yarım Pansiyon High     58.38    100
                            Low      48.58    110
Muğla        Herşey Dahil   High     63.38   9582
                            Low      48.30    237
             Oda + Kahvaltı High     61.26    592
                            Low      37.77     62
             Yarım Pansiyon High     45.82    177
                            Low      32.68     10
İzmir        Herşey Dahil   High     74.75   1234
                            Low      74.31    147
             Oda + Kahvaltı High     54.14     86
                            Low      33.56    142
             Yarım Pansiyon High     87.66    118
                            Low      55.35    777

In [17]:
# Şehir-Concept-CInday kırılımında ücret ortalamaları
df.groupby(["SaleCityName","ConceptName","CInDay"]).agg({"Price": ["mean","count"]})

Price      
                                       mean count
SaleCityName ConceptName    CInDay               
Antalya      Herşey Dahil   Friday    62.66  4136
                            Monday    63.26  6831
                            Saturday  64.42  4741
                            Sunday    65.85  3818
                            Thursday  62.89  3898
...                                     ...   ...
İzmir        Yarım Pansiyon Saturday  52.50   315
                            Sunday    98.93    68
                            Thursday  52.48    91
                            Tuesday   56.14    73
                            Wednesday 54.06    86

[126 rows x 2 columns]

GÖREV 4: City-Concept-Season kırılımın çıktısını PRICE'a göre sıralayınız.

In [18]:
agg_df=df.groupby(["SaleCityName","ConceptName","Seasons"]).agg({"Price": "mean"}).sort_values("Price",ascending=False)
agg_df.head(20)

Price
SaleCityName ConceptName    Seasons       
Girne        Herşey Dahil   High    103.94
                            Low      90.94
İzmir        Yarım Pansiyon High     87.66
Diğer        Herşey Dahil   Low      87.31
                            High     83.79
İzmir        Herşey Dahil   High     74.75
                            Low      74.31
Antalya      Yarım Pansiyon High     73.26
             Oda + Kahvaltı High     66.27
             Herşey Dahil   High     64.92
Muğla        Herşey Dahil   High     63.38
Antalya      Yarım Pansiyon Low      62.00
             Herşey Dahil   Low      61.55
Muğla        Oda + Kahvaltı High     61.26
Antalya      Oda + Kahvaltı Low      60.67
Girne        Yarım Pansiyon High     58.38
İzmir        Yarım Pansiyon Low      55.35
Aydın        Herşey Dahil   High     54.95
İzmir        Oda + Kahvaltı High     54.14
Girne        Yarım Pansiyon Low      48.58

GÖREV 5: Indekste yer alan isimleri değişken ismine çeviriniz.


In [19]:
# Üçüncü sorunun çıktısında yer alan PRICE dışındaki tüm değişkenler index isimleridir.
# Bu isimleri değişken isimlerine çeviriniz.
# İpucu: reset_index()
#yukarıdaki görüntüyü düzenleyelim.
#kalıcı değişiklik: inplace=True
agg_df.reset_index(inplace=True)
agg_df.head()

,SaleCityName,ConceptName,Seasons,Price
0,Girne,Herşey Dahil,High,103.94
1,Girne,Herşey Dahil,Low,90.94
2,İzmir,Yarım Pansiyon,High,87.66
3,Diğer,Herşey Dahil,Low,87.31
4,Diğer,Herşey Dahil,High,83.79


In [20]:
#yeni değişken= 3 değişken seç.agg içerisine x axis=1 yani değişkenlerde gezsin_ ile join etsin ve upper ile büyük harf

In [21]:
agg_df['sales_level_based'] = agg_df[["SaleCityName", "ConceptName", "Seasons"]].agg(lambda x: '_'.join(x).upper(), axis=1)


In [22]:
agg_df[["SaleCityName", "ConceptName", "Seasons"]].head()

,SaleCityName,ConceptName,Seasons
0,Girne,Herşey Dahil,High
1,Girne,Herşey Dahil,Low
2,İzmir,Yarım Pansiyon,High
3,Diğer,Herşey Dahil,Low
4,Diğer,Herşey Dahil,High


In [23]:
agg_df.head()

,SaleCityName,ConceptName,Seasons,Price,sales_level_based
0,Girne,Herşey Dahil,High,103.94,GIRNE_HERŞEY DAHIL_HIGH
1,Girne,Herşey Dahil,Low,90.94,GIRNE_HERŞEY DAHIL_LOW
2,İzmir,Yarım Pansiyon,High,87.66,İZMIR_YARIM PANSIYON_HIGH
3,Diğer,Herşey Dahil,Low,87.31,DIĞER_HERŞEY DAHIL_LOW
4,Diğer,Herşey Dahil,High,83.79,DIĞER_HERŞEY DAHIL_HIGH


In [25]:
# GÖREV 7: Personaları segmentlere ayırınız.
# PRICE'a göre segmentlere ayırınız,
# segmentleri "SEGMENT" isimlendirmesi ile agg_df'e ekleyiniz
# segmentleri betimleyiniz

In [26]:
#fiyat aralıgını bilmediğimiz için qcut q ceyrek olup fiyat aralıgını kendisi belirliyor.
agg_df["SEGMENT"] = pd.qcut(agg_df["Price"], 4, labels=["D", "C", "B", "A"])
agg_df.head(30)


,SaleCityName,ConceptName,Seasons,Price,sales_level_based,SEGMENT
0,Girne,Herşey Dahil,High,103.94,GIRNE_HERŞEY DAHIL_HIGH,A
1,Girne,Herşey Dahil,Low,90.94,GIRNE_HERŞEY DAHIL_LOW,A
2,İzmir,Yarım Pansiyon,High,87.66,İZMIR_YARIM PANSIYON_HIGH,A
3,Diğer,Herşey Dahil,Low,87.31,DIĞER_HERŞEY DAHIL_LOW,A
4,Diğer,Herşey Dahil,High,83.79,DIĞER_HERŞEY DAHIL_HIGH,A
5,İzmir,Herşey Dahil,High,74.75,İZMIR_HERŞEY DAHIL_HIGH,A
6,İzmir,Herşey Dahil,Low,74.31,İZMIR_HERŞEY DAHIL_LOW,A
7,Antalya,Yarım Pansiyon,High,73.26,ANTALYA_YARIM PANSIYON_HIGH,A
8,Antalya,Oda + Kahvaltı,High,66.27,ANTALYA_ODA + KAHVALTI_HIGH,A
9,Antalya,Herşey Dahil,High,64.92,ANTALYA_HERŞEY DAHIL_HIGH,B


In [27]:
#segment kırılımında price ı gözlemleriz, ceyreğe göre bölünen aralıkları daha anlamlı kılar.
agg_df.groupby("SEGMENT").agg({"Price": ["mean", "max", "sum"]})


Price              
         mean    max    sum
SEGMENT                    
D       33.37  39.48 300.30
C       44.89  54.14 403.99
B       60.27  64.92 542.47
A       82.47 103.94 742.21

Görev 8: Yeni gelen müşterileri sınıflandırıp, ne kadar gelir getirebileceklerini tahmin ediniz.

In [28]:
agg_df.sort_values(by="Price")


,SaleCityName,ConceptName,Seasons,Price,sales_level_based,SEGMENT
35,Aydın,Yarım Pansiyon,Low,25.27,AYDIN_YARIM PANSIYON_LOW,D
34,Aydın,Oda + Kahvaltı,High,30.39,AYDIN_ODA + KAHVALTI_HIGH,D
33,Aydın,Yarım Pansiyon,High,32.63,AYDIN_YARIM PANSIYON_HIGH,D
32,Muğla,Yarım Pansiyon,Low,32.68,MUĞLA_YARIM PANSIYON_LOW,D
31,İzmir,Oda + Kahvaltı,Low,33.56,İZMIR_ODA + KAHVALTI_LOW,D
30,Aydın,Herşey Dahil,Low,33.68,AYDIN_HERŞEY DAHIL_LOW,D
29,Diğer,Oda + Kahvaltı,High,34.84,DIĞER_ODA + KAHVALTI_HIGH,D
28,Muğla,Oda + Kahvaltı,Low,37.77,MUĞLA_ODA + KAHVALTI_LOW,D
27,Girne,Oda + Kahvaltı,High,39.48,GIRNE_ODA + KAHVALTI_HIGH,D
26,Diğer,Yarım Pansiyon,High,39.73,DIĞER_YARIM PANSIYON_HIGH,C


In [29]:
new_user = "ANTALYA_HERŞEY DAHIL_HIGH"
agg_df[agg_df["sales_level_based"] == new_user]


,SaleCityName,ConceptName,Seasons,Price,sales_level_based,SEGMENT
9,Antalya,Herşey Dahil,High,64.92,ANTALYA_HERŞEY DAHIL_HIGH,B


In [30]:
deneme="GIRNE_YARIM PANSIYON_LOW"
agg_df[agg_df["sales_level_based"] == deneme]

,SaleCityName,ConceptName,Seasons,Price,sales_level_based,SEGMENT
19,Girne,Yarım Pansiyon,Low,48.58,GIRNE_YARIM PANSIYON_LOW,C
